In [ ]:
library(plotly)

In [ ]:
data <- read.csv("~/CLEAN_MensInternationalComplete.csv")

In [ ]:
statTime <- function(stat, dataset, league, timestat, group){
  
  # if loop to reduce dataframe to just one league - based on the league passed in
  if(missing(league)){
    # if there is no league passed in, keep as is 
    league <- "All Data"
    
  } else if(league == "International" | league == "international"){
    dataset <- dataset[dataset$Team == "wales" | dataset$Team == "new-zealand" | dataset$Team == "australia" | dataset$Team == "scotland" | dataset$Team == "samoa" | dataset$Team == "argentina" | dataset$Team == "usa" | dataset$Team == "italy" | dataset$Team == "fiji" | dataset$Team == "france" | dataset$Team == "japan" | dataset$Team == "canada" | dataset$Team == "ireland" | dataset$Team == "england" | dataset$Team == "tonga" | dataset$Team == "south-africa"| dataset$Team == "georgia", ]
    
  } else if(league == "Super Rugby" | league == "super rugby" | league == "Super rugby" | league == "super Rugby"){
    
    dataset <- dataset[dataset$Team == "highlanders" | dataset$Team == "waratahs" | dataset$Team == "rebels" | dataset$Team == "reds" | dataset$Team == "jaguares" | dataset$Team == "chiefs" | dataset$Team == "blues" | dataset$Team == "stormers" | dataset$Team == "hurricanes" | dataset$Team == "sunwolves" | dataset$Team == "brumbies" | dataset$Team == "sharks" | dataset$Team == "bulls" | dataset$Team == "crusaders" | dataset$Team == "lions", ]
    
  } else if(league == "Premiership" | personleague == "premiership"){
    
    dataset <- dataset[dataset$Team == "gloucester" | dataset$Team == "harlequins" | dataset$Team == "sale" | dataset$Team == "exeter-chiefs" | dataset$Team == "northampton" | dataset$Team == "bath" | dataset$Team == "saracens" | dataset$Team == "bristol" | dataset$Team == "worcester" | dataset$Team == "leicester" | dataset$Team == "wasps", ]

  }
  
  
  # subsets the dataframe to just those observations that are specified when the user utilizes the group argument in the function
  if(missing(group)){
    # the dataframe stays the same if no group arguement is passed in
    dataset <- dataset
    # Set the group for the visualization title
    group <- "All Players"
    # If the group contains position numbers, the dataframe will be subset to those rows with those positions
  } else if(sum(group %in% c(1:15)) > 0){
    dataset <- dataset[dataset$PosNumber %in% group, ]
    # Set the group for the visualization title
    group <- paste("Number", toString(group))
    # If the group argument is "forwards" or a variation, the dataframe is subset to those rows with those positional players
  } else if(group == "forwards" | group == "forward" | group == "Forwards" | group == "Forward"){
    dataset <- dataset[dataset$PosNumber == 1 | dataset$PosNumber == 2 | dataset$PosNumber == 3 | dataset$PosNumber == 4 | dataset$PosNumber == 5 | dataset$PosNumber == 6 | dataset$PosNumber == 7 | dataset$PosNumber == 8, ]
    # Set the group for the visualization title
    group <- "Forwards"
    # If the group argument is "backs" or a variation, the dataframe is subset to those rows with those positional players
  } else if(group == "backs" | group == "back" | group == "Backs" | group == "Back"){
    dataset <- dataset[dataset$PosNumber == 9 | dataset$PosNumber == 10 | dataset$PosNumber == 11 | dataset$PosNumber == 12 | dataset$PosNumber == 13 | dataset$PosNumber == 14 | dataset$PosNumber == 15, ]
    # Set the group for the visualization title
    group <- "Backs"
    # If none of the other conditions are satisfied, an error is printed
  } else {
    return("Invalid group.")
  }
  
  
  
  # if loop for the optional timestat arguement - if the user chooses to normalize the stat by minutes played or games played, it will assess the percentile by dividing the stat by time played
  if(missing(timestat)){
    dataset <- dataset # if no timestat is passed in, the dataframe stays the same
    
    # if "Minutes" is passed in, the dataframe for comparison and the player dataframe are reduced to just those rows that have time information  
  } else if(timestat == "Minutes"){
    dataset <- dataset[complete.cases(dataset[ , which(names(dataset) == "Minutes.Played")]) ,]
    dataset <- dataset[dataset[ , which(names(dataset) == "Minutes.Played")] > 0, ]
    
    # add another column, which divides the stat by time played, is asses to the dataframe and the person dataframe
    dataset$statPTime <- dataset[ , which(names(dataset) == stat)] / dataset[ , which(names(dataset) == "Minutes.Played")]
    
    # rename the stat argument to the stat/Minute
    stat <- paste0(stat, "/Minute")
    
    # rename the new stat columns to the new stat name, normalized by the time
    names(dataset)[length(dataset)] <- stat
    
    # if "Games" is passed in, the dataframe for comparison and the player dataframe are reduced to just those rows that have time information  
  } else if(timestat == "Games"){
    dataset <- dataset[complete.cases(dataset[ , which(names(dataset) == "Games")]) ,]
    dataset <- dataset[dataset[ , which(names(dataset) == "Games")] > 0,]

    
    # add another column, which divides the stat by time played, is asses to the dataframe and the person dataframe
    dataset$statPTime <- dataset[ , which(names(dataset) == stat)] / dataset[ , which(names(dataset) == "Games")]
    
    # rename the stat argument to the stat/Minute
    stat <- paste0(stat, "/Game")
    
    # rename the new stat columns to the new stat name, normalized by the time
    names(dataset)[length(dataset)] <- stat
  }

  # create a new dataset that has a row for each year and the mean stat for each year
  graphSet <- as.data.frame(matrix(ncol = 2, nrow = length(unique(dataset$Year))))
  graphSet[,1] <- sort(unique(dataset$Year))
  for(i in 1:length(unique(dataset$Year))){
    graphSet[i,2] <- mean(dataset[dataset$Year == graphSet[i,1], which(names(dataset) == stat)], na.rm = TRUE)
  }
  names(graphSet) <- c("Year", stat)
  
  p <- plot_ly(x = graphSet[, 1], y = graphSet[, 2], type = "scatter" , mode = "lines", showlegend = FALSE) %>%
    add_markers() %>%
    layout(title = paste0(league, ", ", stat, " Over Time, ", group),
           scene = list(xaxis = list(title = "Year"),
                        yaxis = list(title = stat)))
  return(p)
}

In [ ]:
statTime(stat = "Tackles", dataset = data)

In [ ]:
statTime(stat = "Tackles", dataset = data, league = "International", timestat = "Games")

In [ ]:
statTime(stat = "Tackles", dataset = data, league = "International", group = "backs")

In [ ]:
statTime(stat = "Tackles", dataset = data, league = "International", group = c(6,7,8))